In [62]:
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import svm
import numpy as np
import pandas as pd

In [63]:
# lendo dados do prof
treino = pd.read_csv("../treinamento.txt", sep=' ', header=None)
classes_treino = treino[treino.columns[-1]].to_numpy()
# print(classes_treino)
nlinhas = len(treino)
treino = treino.iloc[:,:-1].to_numpy()

teste = pd.read_csv("../teste.txt", sep=' ', header=None)
classes_teste = teste[teste.columns[-1]].to_numpy()
nlinhas_teste = len(teste)
teste = teste.iloc[:,:-1].to_numpy()

# lendo dados do sklearn
dados, classes = load_digits(return_X_y=True)


In [64]:
dados_prof = False
dados_prof = True
classes_possiveis = []
x_train, x_test, y_train, y_test = [],[],[],[]
if dados_prof:
    classes_possiveis = list(set(classes_teste))
    
    x, y = treino, classes_treino
    for c in classes_possiveis:
        idxs: np.ndarray = y == c
        x_train.append(x[idxs,:])
        y_train.extend(y[idxs])
    x_train = np.concatenate(x_train)
    y_train = np.array(y_train)

    x, y = teste, classes_teste

    for c in classes_possiveis:
        idxs: np.ndarray = y == c
        x_test.append(x[idxs,:])
        y_test.extend(y[idxs])
    x_test = np.concatenate(x_test)
    y_test = np.array(y_test)
else:
    classes_possiveis = list(set(classes))
    x, y = dados, classes
    xc = []
    yc = []
    for c in classes_possiveis:
        idxs: np.ndarray = y == c
        xc.append(x[idxs,:])
        yc.extend(y[idxs])
    xc = np.concatenate(xc)
    yc = np.array(yc)
    x_train, x_test, y_train, y_test = train_test_split(xc, yc, test_size=0.2)

In [65]:
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

In [69]:
km = KMeans(n_clusters=10, n_init=10)
centroides_classes = {}
for digit_class in classes_possiveis:
    indexes = y_train == digit_class
    instancias = x_train[indexes]
    km.fit(instancias)
    centroides_classes[digit_class] = km.cluster_centers_
centroides_para_treino = []
centroides_labels = []
for i in centroides_classes:
    for centroide in centroides_classes[i]:
        centroides_para_treino.append(centroide)
        centroides_labels.append(i)
    


In [81]:
modelo = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
modelo.fit(centroides_para_treino, centroides_labels)
print(modelo.score(x_test, y_test))
print(classification_report(y_test, modelo.predict(x_test)))
print("-------------------------------------------")
srv = svm.SVC(probability=True, kernel='linear')
#srv.fit(x_train, y_train)
srv.fit(centroides_para_treino, centroides_labels)
print(srv.score(x_test, y_test))
print(classification_report(y_test, srv.predict(x_test)))

0.885
              precision    recall  f1-score   support

           0       0.77      0.97      0.86       100
           1       0.82      0.88      0.85       100
           2       0.90      0.87      0.88       100
           3       0.91      0.91      0.91       100
           4       0.91      0.96      0.93       100
           5       0.95      0.91      0.93       100
           6       0.94      0.98      0.96       100
           7       0.93      0.95      0.94       100
           8       0.85      0.58      0.69       100
           9       0.89      0.84      0.87       100

    accuracy                           0.89      1000
   macro avg       0.89      0.89      0.88      1000
weighted avg       0.89      0.89      0.88      1000

-------------------------------------------
0.945
              precision    recall  f1-score   support

           0       0.92      0.97      0.94       100
           1       0.89      0.93      0.91       100
           2       0.9